In [4]:
API_KEY = "voc-232586434104020301899465f0c2fdcc1e25.50297608"
BASE_URL = "https://openai.vocareum.com/v1"

In [5]:
from openai import OpenAI

In [6]:
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

In [54]:
from openai import OpenAI

completion = client.chat.completions.create(
  # model="gpt-4",
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a programming assistant with a specialization in logic programming and Prolog. You can interpret complex scenarios and model them in Prolog, and you understand how to verify the correctness of logical assertions using queries."},
    {"role": "user", "content": "convert to prolog and check whether the answer is correct \n I would like the all the lines of code, except the last line, to be run in a .pl file and the last line of code to be a query to validate the answer \n Please do not answer anything other than code \n context: There was a group discussion of judicial workers in the city.One group has 8 people.At the beginning of the meeting, the group leader asked everyone if they knew each other.As a result, only one person in the group knew 3 of the group, 3 knew 2 of the group, and 4 knew 1 of the group. \n question: If the above statistics are true, which of the following conclusions can best be reached?\n answer: C.Some members may only know what they have seen on television or at a briefing \n Here is an example: \n context: Some Cantonese don't like chili, so some southerners don't like chili. \n question:Which of the following can guarantee the above argument? \n answer:D.Some Cantonese like neither peppers nor sweets \n % Define that there is at least one Cantonese who doesn't like chili \n cantonese(X) :- dislike_chili(X). \n % If a person is Cantonese and dislikes chili, then there is at least one Southern Chinese who dislikes chili \n southern_chinese(Y) :- cantonese(Y), dislike_chili(Y). \n % Define that there is at least one Cantonese who likes neither peppers nor sweets \n likes_neither_peppers_nor_sweets(X) :- cantonese(X), not(likes_peppers(X)), not(likes_sweets(X)). \n % Check if the above statement implies that there is at least one Southern Chinese who dislikes chili \n % (Assuming that liking neither peppers nor sweets implies not liking chili) \n guarantee_statement(Y) :- likes_neither_peppers_nor_sweets(Y), dislike_chili(Y). \n % Facts (assuming we have someone who meets the criteria for the answer) \n cantonese(sam). \n not(likes_peppers(sam)). \n not(likes_sweets(sam)). \n ?- guarantee_statement(Who)."}
  ]
)

# print(completion.choices[0].message)

answer = completion.choices[0].message


In [55]:
print(answer)

ChatCompletionMessage(content='cantonese(X) :- dislike_chili(X).\nsouthern_chinese(Y) :- cantonese(Y), dislike_chili(Y).\nlikes_neither_peppers_nor_sweets(X) :- cantonese(X), \\+ likes_peppers(X), \\+ likes_sweets(X).\nguarantee_statement(Y) :- likes_neither_peppers_nor_sweets(Y), dislike_chili(Y).\n\ncantonese(sam).\n\\+ likes_peppers(sam).\n\\+ likes_sweets(sam).\n\n?- guarantee_statement(Who).', role='assistant', function_call=None, tool_calls=None)


In [56]:
message_content = answer.content
answer_lines = message_content.split('\n')

code_lines = answer_lines[:-1]
query_line = answer_lines[-1]

# Write code lines to 'test1_gpt3.5.pl' file
with open('test1_gpt3.5.pl', 'w') as file:
    for line in code_lines:
        file.write(line + '\n')

# Append query line to 'query' file
with open('query', 'a') as file:
    file.write(query_line + '\n')

In [57]:
def get_prompt(df):
    context = df.context
    question = df.question
    answer = df.Answer
    return f"""convert to prolog and check whether the answer is correct \n I would like the all the lines of code, except the last line, to be run in a .pl file and the last line of code to be a query to validate the answer \n Please do not answer anything other than code \n context: {context} \n question: {question} \n answer: {answer} \n Here is an example: \n context: Some Cantonese don't like chili, so some southerners don't like chili. \n question:Which of the following can guarantee the above argument? \n answer:D.Some Cantonese like neither peppers nor sweets \n % Define that there is at least one Cantonese who doesn't like chili \n cantonese(X) :- dislike_chili(X). \n % If a person is Cantonese and dislikes chili, then there is at least one Southern Chinese who dislikes chili \n southern_chinese(Y) :- cantonese(Y), dislike_chili(Y). \n % Define that there is at least one Cantonese who likes neither peppers nor sweets \n likes_neither_peppers_nor_sweets(X) :- cantonese(X), not(likes_peppers(X)), not(likes_sweets(X)). \n % Check if the above statement implies that there is at least one Southern Chinese who dislikes chili \n % (Assuming that liking neither peppers nor sweets implies not liking chili) \n guarantee_statement(Y) :- likes_neither_peppers_nor_sweets(Y), dislike_chili(Y). \n % Facts (assuming we have someone who meets the criteria for the answer) \n cantonese(sam). \n not(likes_peppers(sam)). \n not(likes_sweets(sam)). \n ?- guarantee_statement(Who)."""

In [58]:
import pandas as pd

In [60]:
test = pd.read_csv('dataset/train.csv')
with open('gpt3.5_prolog/query', 'w') as file:
    pass  # Simply pass, which empties the file
# for i in range(len(test)):
for i in range(30):
    prompt = get_prompt(test.iloc[i])
    completion = client.chat.completions.create(
        model="gpt-4",
        # model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a programming assistant with a specialization in logic programming and Prolog. You can interpret complex scenarios and model them in Prolog, and you understand how to verify the correctness of logical assertions using queries."},
            {"role": "user",
             "content": prompt}
        ]
    )
    answer = completion.choices[0].message
    message_content = answer.content
    answer_lines = [line for line in message_content.split('\n') if '```' not in line and line.strip()]

    code_lines = answer_lines[:-1]
    query_line = answer_lines[-1]

    filename = f"gpt3.5_prolog/test{i+1}_gpt3.5.pl"
    with open(filename, 'w') as file:
        for line in code_lines:
            file.write(line + '\n')

    with open('gpt3.5_prolog/query', 'a') as file:
        file.write(query_line + '\n')

In [61]:
import pandas as pd

predictions = []
result_file_path = 'gpt3.5_prolog/prolog_results.txt'
label_file_path = 'dataset/train.csv'
with open(result_file_path, 'r') as file:
    for line in file:
        predictions.append(int(line.strip()))

test = pd.read_csv(label_file_path)
labels = test['Label'].tolist()[:len(predictions)]

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
f1 = f1_score(labels, predictions)
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
print(f'F1: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}')

F1: 0.717948717948718, Accuracy: 0.5925925925925926, Precision: 0.5833333333333334, Recall: 0.9333333333333333
